# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing GraphLab functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    
## Fire up GraphLab Create

Make sure you have the latest version of GraphLab Create.

In [52]:
from __future__ import division
import graphlab
import math
import string

# Data preparation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [2]:
products = graphlab.SFrame('amazon_baby.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to tor.h.halvorsen@gmail.com and will expire on April 15, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\torhh\AppData\Local\Temp\graphlab_server_1492385276.log.0


Now, let us see a preview of what the dataset looks like.

In [3]:
products

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [8]:
products[269]

{'name': 'The First Years Massaging Action Teether',
 'rating': 5.0,
 'review': 'A favorite in our house!',
 'word_count': {'a': 1L, 'favorite': 1L, 'house': 1L, 'in': 1L, 'our': 1L}}

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [7]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

review_without_punctuation = products['review'].apply(remove_punctuation)
products['word_count'] = graphlab.text_analytics.count_words(review_without_punctuation)

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [6]:
products[269]['word_count']

{'a': 1L, 'favorite': 1L, 'house': 1L, 'in': 1L, 'our': 1L}

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [9]:
products = products[products['rating'] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [10]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3L, 'love': 1L,'it': 3L, 'highly': 1L, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2L, 'quilt': 1L,'it': 1L, 'comfortable': ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'and': 3L, 'ingenious':1L, 'love': 2L, 'is': ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2L, 'all': 2L,'help': 1L, 'cried': 1L, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2L, 'cute': 1L,'help': 2L, 'habit': 1L, ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1L, 'be': 1L,'is': 1L, 'bound': 1L, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2L, 'all': 1L,'right': 1L, 'able': 1L, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1L, 'fantastic':1L, 'help': 1L, 'give': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'all': 1L, 'standarad':1L, 'another': 1L, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 2L, 'nannys': 1L,'just': 1L, 'sleep': 2L, ...",1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [11]:
train_data, test_data = products.random_split(.8, seed=1)
print len(train_data)
print len(test_data)

133416
33336


# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**Note:** This line may take 1-2 minutes.

In [12]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                      target = 'sentiment',
                                                      features=['word_count'],
                                                      validation_set=None)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 12       | 3.000000  | 5.123188     | 0.960964          |

| 6         | 13       | 3.000000  | 5.570373     | 0.975033          |

+-----------+----------+-----------+--------------+-------------------+

TERMINATED: Terminated due to numerical difficulties.

This model may not be ideal. To improve it, consider doing one of the following:
(a) Increasing the regularization.
(b) Standardizing the input data.
(c) Removing highly correlated features.
(d) Removing `inf` and `NaN` values in the training data.

In [13]:
sentiment_model

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 121713
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 121712

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : lbfgs
Solver iterations              : 6
Solver status                  : TERMINATED: Terminated due to numerical difficulties.
Training time (sec)            : 6.012

Settings
--------
Log-likelihood                 : inf

Highest Positive Coefficients
-----------------------------
word_count[mobileupdate]       : 41.9847
word_count[placeid]            : 41.7354
word_count[labelbox]           : 41.151
word_count[httpwwwamazoncomreviewrhgg6qp7tdnhbrefcmcrprcmtieutf8asinb00318cla0nodeid] : 40.0454
word_count[knobskeeping]       : 36.2091

Lowest Negative Coeffic

**Aside**. You may get a warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

In [14]:
weights = sentiment_model.coefficients
weights.column_names()

['name', 'index', 'class', 'value', 'stderr']

There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).


In [30]:
num_positive_weights = 0
num_negative_weights = 0

num_coeff = 0

print(weights)
print(len(weights))

while num_coeff < len(weights):
    if(weights[num_coeff]['value'] >= 0):
        num_positive_weights = num_positive_weights + 1
    else:
        num_negative_weights = num_negative_weights + 1 
    num_coeff = num_coeff +1

print(num_coeff)
print(num_negative_weights + num_positive_weights)
print "Number of positive weights: %s " % num_positive_weights
print "Number of negative weights: %s " % num_negative_weights

+-------------+--------------+-------+------------------+--------+
|     name    |    index     | class |      value       | stderr |
+-------------+--------------+-------+------------------+--------+
| (intercept) |     None     |   1   |  1.30337080544   |  None  |
|  word_count |  recommend   |   1   |  0.303815600015  |  None  |
|  word_count |    highly    |   1   |  1.49183015276   |  None  |
|  word_count | disappointed |   1   |  -3.95748618393  |  None  |
|  word_count |     love     |   1   |  1.43301685439   |  None  |
|  word_count |      it      |   1   | 0.00986646490307 |  None  |
|  word_count |    planet    |   1   | -0.797764553926  |  None  |
|  word_count |     and      |   1   |  0.048449573172  |  None  |
|  word_count |     bags     |   1   |  0.165541436615  |  None  |
|  word_count |    wipes     |   1   | -0.0949937947269 |  None  |
+-------------+--------------+-------+------------------+--------+
[121713 rows x 5 columns]
Note: Only the head of the SFrame is

**Quiz Question:** How many weights are >= 0?

## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [31]:
sample_test_data = test_data[10:13]
print sample_test_data['rating']
sample_test_data

[5.0, 2.0, 1.0]


name,review,rating,word_count,sentiment
Our Baby Girl Memory Book,Absolutely love it andall of the Scripture in ...,5.0,"{'and': 2L, 'all': 1L,'love': 1L, ...",1
Wall Decor RemovableDecal Sticker - Colorful ...,Would not purchase againor recommend. The decals ...,2.0,"{'and': 1L, 'wall': 1L,'them': 1L, 'decals': ...",-1
New Style Trailing CherryBlossom Tree Decal ...,Was so excited to getthis product for my baby ...,1.0,"{'all': 1L, 'money': 1L,'into': 1L, 'it': 3L, ...",-1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [32]:
sample_test_data[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [33]:
sample_test_data[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using GraphLab Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [34]:
scores = sentiment_model.predict(sample_test_data, output_type='margin')
print scores

[6.734619727059237, -5.734130996760289, -14.668460404468602]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [48]:
def predict_sentiment(data, output):
    i=0
    length = len(data)
    #print("Length = " +str(length) )
    

    while( i < length ):
        #print(i)
        if( data[i] > 0):
            output.append(1)
        else:
            output.append(-1)
        i = i + 1
#%%

tor_output = []
print(scores)
predict_sentiment(scores, tor_output)
print(tor_output)


[6.734619727059237, -5.734130996760289, -14.668460404468602]
[1, -1, -1]


Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from GraphLab Create.

In [49]:
print "Class predictions according to GraphLab Create:" 
print sentiment_model.predict(sample_test_data)

Class predictions according to GraphLab Create:
[1L, -1L, -1L]


**Checkpoint**: Make sure your class predictions match with the one obtained from GraphLab Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [56]:
def predict_probability(data, output):
    i=0
    length = len(data)
    #print("Length = " +str(length) )
    
    
    while( i < length ):
        #print(i)
        numerator = 1
        denumerator = 1 + math.exp(data[i]*-1)
        #print(denumerator)
        output.append(numerator / denumerator)
        i = i + 1
#%%

tor_output = []
print(scores)
predict_probability(scores, tor_output)
print(tor_output)

[6.734619727059237, -5.734130996760289, -14.668460404468602]
[0.9988123848377196, 0.0032232681818005803, 4.261557996655065e-07]


**Checkpoint**: Make sure your probability predictions match the ones obtained from GraphLab Create.

In [57]:
print "Class predictions according to GraphLab Create:" 
print sentiment_model.predict(sample_test_data, output_type='probability')

Class predictions according to GraphLab Create:
[0.9988123848377197, 0.003223268181800582, 4.261557996655062e-07]


** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use GraphLab Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [136]:
probability_test = sentiment_model.predict(test_data, output_type='probability')
print(probability_test)
sf_tor = graphlab.SFrame({'index': range(len(probability_test))})
#sf_tor = graphlab.SFrame({"index":[int()], "value":[float()]})
print(sf_tor.column_types())
#sf_tor['value'] = sf_tor['value'].astype(float)
sf_tor['value'] = probability_test.astype(float)
print(sf_tor.column_types())
print(sf_tor)
#print(test_data)
top = 20
i=0

sf_tor_top = graphlab.SFrame({'i': [int()], 'v':[float()]})
#sf_tor['value'] = probability_test.astype(float)
print(sf_tor_top.column_types())
print(sf_tor_top)
print("HI - 1")
sf_tor_top = sf_tor.topk('value',top)

print(sf_tor_top.column_types())
print(sf_tor_top)
print("HI - 2")
#sf_tor = sf_tor.topk('value',top)
#print(sf_tor_top.column_types())
#print(sf_tor_top)

#sf_tor_top.print_rows(top)
#sf_tor.print_rows(top)
#print(sf_tor[130])
#print(sf_tor_top['index' == 130])
#print(sf_tor[9116])
#print(sf_tor[564])
#print(sf_tor[10170])
while(i < top):
    print "Index: " +  str(sf_tor_top[i]['index']) + " Value: " + str(sf_tor_top[i]['value']),
    print("Name: " + test_data[sf_tor_top[i]['index']]['name'])
    i = i +1
#(sf_tor.topk('value',20))

[0.7583998877521196, 0.9999999999664292, 0.22895097808037593, 0.9999995580631229, 0.990542169248041, 0.9999992959681095, 0.9997644762803491, 0.7228344662832066, 0.9992668408960922, 0.9997868300478303, 0.9988123848377197, 0.003223268181800582, 4.261557996655062e-07, 0.9990418823663026, 0.9987309104296942, 0.9906796358748918, 0.9998406264765501, 0.1316444167364354, 0.0666033706633921, 0.9999031091993139, 0.9993746129277777, 0.9995211667719665, 0.999999999909259, 0.9999997936706093, 0.9999999270043358, 0.9965177508089255, 0.9921301669166996, 0.998056525789808, 0.9999999999977496, 0.999984481164085, 0.9988128229682436, 0.9999801239311171, 0.9999981532032528, 0.9999991979779302, 0.9999999671985775, 0.9999310908183666, 0.9999333610274893, 0.999210749001976, 0.9786387951021511, 0.9999598797394904, 0.928146407681726, 0.9985460653233494, 0.8523971428467374, 0.9946850076384276, 0.9993432830032861, 0.9996741991368743, 0.9997851666395846, 0.8928167682318845, 0.022952966830242807, 0.999988026317077

**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [163]:
probability_test = sentiment_model.predict(test_data, output_type='probability')
#print(probability_test)
sf_tor = graphlab.SFrame({'index': range(len(probability_test))})
sf_tor['value'] = probability_test
#print(sf_tor)
#print(test_data)
top = 20
i=0
sf_tor_top = sf_tor.sort('value', ascending = True)

#sf_tor_top.print_rows(20)
#print(sf_tor[1497])
#print(sf_tor[0])
while(i < top):
    print "Index: " +  str(sf_tor_top[i]['index']) + " Value: " + str(sf_tor_top[i]['value']),
    print("Name: " + test_data[sf_tor_top[i]['index']]['name'])
    i = i +1
#(sf_tor.topk('value',20))

Index: 25483 Value: 7.80415068207e-100 Name: Jolly Jumper Arctic Sneak A Peek Infant Car Seat Cover Black
Index: 21700 Value: 6.83650885514e-25 Name: Levana Safe N'See Digital Video Baby Monitor with Talk-to-Baby Intercom and Lullaby Control (LV-TW501)
Index: 17034 Value: 2.12654510823e-24 Name: Snuza Portable Baby Movement Monitor
Index: 2931 Value: 2.24582080779e-23 Name: Fisher-Price Ocean Wonders Aquarium Bouncer
Index: 28184 Value: 1.32962966148e-22 Name: VTech Communications Safe &amp; Sounds Full Color Video and Audio Monitor
Index: 9655 Value: 2.06872097469e-20 Name: Safety 1st High-Def Digital Monitor
Index: 7310 Value: 5.93881994671e-20 Name: Chicco Cortina KeyFit 30 Travel System in Adventure
Index: 13572 Value: 6.28510016539e-20 Name: Prince Lionheart Warmies Wipes Warmer
Index: 17985 Value: 8.0552871269e-20 Name: Valco Baby Tri-mode Twin Stroller EX- Hot Chocolate
Index: 8818 Value: 8.46521724942e-20 Name: Adiri BPA Free Natural Nurser Ultimate Bottle Stage 1 White, Slow F

**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [150]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    ## YOUR CODE HERE
    predictions = model.predict(data, output_type="probability")
    print(predictions)
    print("true_labels")
    print(true_labels)
    
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    n = 0
    correct = 0
    stop = 4
    stop = len(data)
    while( n < stop):
        if(predictions[n] > 0.5):
            if( true_labels[n] == 1):
                correct = correct +1
        else:
            if( true_labels[n] == -1):
                correct = correct +1
        n = n +1

    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    accuracy = correct / stop
    
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [151]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

[0.7583998877521196, 0.9999999999664292, 0.22895097808037593, 0.9999995580631229, 0.990542169248041, 0.9999992959681095, 0.9997644762803491, 0.7228344662832066, 0.9992668408960922, 0.9997868300478303, 0.9988123848377197, 0.003223268181800582, 4.261557996655062e-07, 0.9990418823663026, 0.9987309104296942, 0.9906796358748918, 0.9998406264765501, 0.1316444167364354, 0.0666033706633921, 0.9999031091993139, 0.9993746129277777, 0.9995211667719665, 0.999999999909259, 0.9999997936706093, 0.9999999270043358, 0.9965177508089255, 0.9921301669166996, 0.998056525789808, 0.9999999999977496, 0.999984481164085, 0.9988128229682436, 0.9999801239311171, 0.9999981532032528, 0.9999991979779302, 0.9999999671985775, 0.9999310908183666, 0.9999333610274893, 0.999210749001976, 0.9786387951021511, 0.9999598797394904, 0.928146407681726, 0.9985460653233494, 0.8523971428467374, 0.9946850076384276, 0.9993432830032861, 0.9996741991368743, 0.9997851666395846, 0.8928167682318845, 0.022952966830242807, 0.999988026317077

0.9145368370530358

**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [152]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [153]:
len(significant_words)

20

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

In [154]:
train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)

Let's see what the first example of the dataset looks like:

In [155]:
train_data[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

The **word_count** column had been working with before looks like the following:

In [156]:
print train_data[0]['word_count']

{'and': 3L, 'love': 1L, 'it': 3L, 'highly': 1L, 'osocozy': 1L, 'bags': 1L, 'leak': 1L, 'moist': 1L, 'does': 1L, 'recommend': 1L, 'was': 1L, 'wipes': 1L, 'disappointed': 1L, 'early': 1L, 'not': 2L, 'now': 1L, 'holder': 1L, 'wipe': 1L, 'keps': 1L, 'wise': 1L, 'i': 1L, 'planet': 1L, 'my': 2L, 'came': 1L}


Since we are only working with a subset of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

In [157]:
print train_data[0]['word_count_subset']

{'love': 1L, 'disappointed': 1L}


## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [158]:
simple_model = graphlab.logistic_classifier.create(train_data,
                                                   target = 'sentiment',
                                                   features=['word_count_subset'],
                                                   validation_set=None)
simple_model

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 6        | 0.700381     | 0.866815          |

| 6         | 7        | 0.811180     | 0.866815          |

+-----------+----------+--------------+-------------------+

SUCCESS: Optimal solution found.

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 21
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 20

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 6
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.8408

Settings
--------
Log-likelihood                 : 44323.7254

Highest Positive Coefficients
-----------------------------
word_count_subset[loves]       : 1.6773
word_count_subset[perfect]     : 1.5145
word_count_subset[love]        : 1.3654
(intercept)                    : 1.2995
word_count_subset[easy]        : 1.1937

Lowest Negative Coefficients
----------------------------
word_count_subset[disappointed] : -2.3551
wo

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [159]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

[0.8938702194129586, 0.9967580549293198, 0.9602891156024802, 0.9854256834791519, 0.9311391807567753, 0.9868929913204428, 0.9515157119069159, 0.5157918730110549, 0.9736522600990778, 0.9434293521597004, 0.9349283210437559, 0.6398992894266222, 0.12328661213292642, 0.9604154664811644, 0.9159128679159048, 0.972520993407962, 0.7857583897873343, 0.8605869553050395, 0.2569895579832383, 0.8977665698482419, 0.7935325961771249, 0.9928389001758271, 0.9999728497270532, 0.9635373809104096, 0.9596600647562967, 0.8430860155424402, 0.8090387264038901, 0.9216570970888968, 0.9857200569486576, 0.8817468071641894, 0.972520993407962, 0.7857583897873343, 0.9894488068687575, 0.9900328207082058, 0.9821353261205432, 0.9434293521597004, 0.9635373809104096, 0.972520993407962, 0.7857583897873343, 0.9236642209150074, 0.945649529364673, 0.9688681801439984, 0.7997833973400169, 0.9041522028015928, 0.7857583897873343, 0.9531952687851147, 0.9576601486556942, 0.7857583897873343, 0.4262609384691384, 0.9994993934873903, 0.

0.8693004559635229

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [160]:
simple_model.coefficients

name,index,class,value,stderr
(intercept),None,1,1.2995449552,0.0120888541331
word_count_subset,disappointed,1,-2.35509250061,0.0504149888557
word_count_subset,love,1,1.36543549368,0.0303546295109
word_count_subset,well,1,0.504256746398,0.021381300631
word_count_subset,product,1,-0.320555492996,0.0154311321362
word_count_subset,loves,1,1.67727145556,0.0482328275384
word_count_subset,little,1,0.520628636025,0.0214691475665
word_count_subset,work,1,-0.621700012425,0.0230330597946
word_count_subset,easy,1,1.19366189833,0.029288869202
word_count_subset,great,1,0.94469126948,0.0209509926591


Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [161]:
simple_model.coefficients.sort('value', ascending=False).print_rows(num_rows=21)

+-------------------+--------------+-------+-----------------+-----------------+
|        name       |    index     | class |      value      |      stderr     |
+-------------------+--------------+-------+-----------------+-----------------+
| word_count_subset |    loves     |   1   |  1.67727145556  | 0.0482328275384 |
| word_count_subset |   perfect    |   1   |  1.51448626703  |  0.049861952294 |
| word_count_subset |     love     |   1   |  1.36543549368  | 0.0303546295109 |
|    (intercept)    |     None     |   1   |   1.2995449552  | 0.0120888541331 |
| word_count_subset |     easy     |   1   |  1.19366189833  |  0.029288869202 |
| word_count_subset |    great     |   1   |  0.94469126948  | 0.0209509926591 |
| word_count_subset |    little    |   1   |  0.520628636025 | 0.0214691475665 |
| word_count_subset |     well     |   1   |  0.504256746398 |  0.021381300631 |
| word_count_subset |     able     |   1   |  0.191438302295 | 0.0337581955697 |
| word_count_subset |     ol

**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [164]:
get_classification_accuracy(sentiment_model, train_data, train_data['sentiment'])

[0.9993153660153221, 0.9999516636147674, 0.9999983521360369, 0.9999875877660748, 0.9999999990388249, 0.996915678507483, 0.9999771629836253, 0.9999267004407464, 0.9998347884449732, 0.9917659316250662, 0.9984547329205442, 0.9999999623442307, 0.9894238570673445, 0.9999793739171771, 0.9737401415428474, 1.485640853322571e-06, 0.9990165741541662, 0.9996768876696464, 0.9999990371138129, 0.9999500903117506, 0.9416665708785064, 0.9999335354669178, 0.8810636468047205, 0.9999939225729534, 0.9998284137072491, 0.999983942199634, 0.7864017374764495, 0.980298626566354, 0.9998752205365435, 0.9999929871268064, 0.9997232588316814, 0.9983552290811635, 0.9999996324258653, 0.9959187838625282, 0.9999972532882548, 0.9999923158817441, 0.0019109882319978129, 0.9990392801436709, 0.9999856930663219, 0.9996579141090318, 0.9979736672609009, 0.9999811772452816, 0.7864017374764495, 0.9999111268533938, 0.9999999282525436, 0.9999519028435447, 0.9999998133927952, 0.9998205839193256, 0.9999421535664091, 0.99976748411898

0.979440247046831

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [165]:
get_classification_accuracy(simple_model, train_data, train_data['sentiment'])

[0.5768579103404444, 0.7857583897873343, 0.9977430057744843, 0.9889418690904972, 0.8265501069349547, 0.8162258082841543, 0.93358819297551, 0.7269076575431525, 0.974553077532126, 0.9904834132082667, 0.7857583897873343, 0.8086857064973584, 0.7857583897873343, 0.7857583897873343, 0.7857583897873343, 0.2581612435637129, 0.9849240937022106, 0.9907582111595281, 0.9328897908211073, 0.7857583897873343, 0.9091310764538938, 0.9905376676253755, 0.5990866195398545, 0.998557813945357, 0.9434293521597004, 0.9772176322794089, 0.7857583897873343, 0.7857583897873343, 0.9159128679159048, 0.9985561610675178, 0.7857583897873343, 0.9531952687851147, 0.9553068605751113, 0.9805736559927538, 0.9825432152288467, 0.9842279596725527, 0.11884412710444851, 0.7997833973400169, 0.8605869553050395, 0.9736522600990778, 0.9531952687851147, 0.7185410058515431, 0.7857583897873343, 0.7857583897873343, 0.9864568835719633, 0.9736522600990778, 0.9434293521597004, 0.9434293521597004, 0.9041522028015928, 0.8686640474701104, 0.

0.8668150746537147

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

Now, we will repeat this exercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [166]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

[0.7583998877521196, 0.9999999999664292, 0.22895097808037593, 0.9999995580631229, 0.990542169248041, 0.9999992959681095, 0.9997644762803491, 0.7228344662832066, 0.9992668408960922, 0.9997868300478303, 0.9988123848377197, 0.003223268181800582, 4.261557996655062e-07, 0.9990418823663026, 0.9987309104296942, 0.9906796358748918, 0.9998406264765501, 0.1316444167364354, 0.0666033706633921, 0.9999031091993139, 0.9993746129277777, 0.9995211667719665, 0.999999999909259, 0.9999997936706093, 0.9999999270043358, 0.9965177508089255, 0.9921301669166996, 0.998056525789808, 0.9999999999977496, 0.999984481164085, 0.9988128229682436, 0.9999801239311171, 0.9999981532032528, 0.9999991979779302, 0.9999999671985775, 0.9999310908183666, 0.9999333610274893, 0.999210749001976, 0.9786387951021511, 0.9999598797394904, 0.928146407681726, 0.9985460653233494, 0.8523971428467374, 0.9946850076384276, 0.9993432830032861, 0.9996741991368743, 0.9997851666395846, 0.8928167682318845, 0.022952966830242807, 0.999988026317077

0.9145368370530358

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [167]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

[0.8938702194129586, 0.9967580549293198, 0.9602891156024802, 0.9854256834791519, 0.9311391807567753, 0.9868929913204428, 0.9515157119069159, 0.5157918730110549, 0.9736522600990778, 0.9434293521597004, 0.9349283210437559, 0.6398992894266222, 0.12328661213292642, 0.9604154664811644, 0.9159128679159048, 0.972520993407962, 0.7857583897873343, 0.8605869553050395, 0.2569895579832383, 0.8977665698482419, 0.7935325961771249, 0.9928389001758271, 0.9999728497270532, 0.9635373809104096, 0.9596600647562967, 0.8430860155424402, 0.8090387264038901, 0.9216570970888968, 0.9857200569486576, 0.8817468071641894, 0.972520993407962, 0.7857583897873343, 0.9894488068687575, 0.9900328207082058, 0.9821353261205432, 0.9434293521597004, 0.9635373809104096, 0.972520993407962, 0.7857583897873343, 0.9236642209150074, 0.945649529364673, 0.9688681801439984, 0.7997833973400169, 0.9041522028015928, 0.7857583897873343, 0.9531952687851147, 0.9576601486556942, 0.7857583897873343, 0.4262609384691384, 0.9994993934873903, 0.

0.8693004559635229

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [ ]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print num_positive
print num_negative

Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?